# Meta-analysis demo (toy dataset)

This notebook creates/loads a tiny CSV, computes fixed- and random-effects pooled estimates (inverse-variance, DerSimonian–Laird), saves a forest plot and summary table into `outputs/`, and appends a run log with timestamps and file paths.


In [ ]:
from pathlib import Path
from datetime import datetime, timezone
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

BASE = Path('/Users/jtr/_JTR23_/COSMO/runtime/outputs/execution')
DATA_DIR = BASE / 'data'
OUT_DIR = BASE / 'outputs'
for d in (DATA_DIR, OUT_DIR):
    d.mkdir(parents=True, exist_ok=True)

csv_path = DATA_DIR / 'toy_extraction.csv'

# Toy dataset: log(OR) with standard error (5–10 rows)
if not csv_path.exists():
    toy = pd.DataFrame({
        'study': ['Study A', 'Study B', 'Study C', 'Study D', 'Study E', 'Study F', 'Study G'],
        'yi':    [-0.20,     0.10,      -0.05,     0.30,      -0.15,     0.05,      0.25],
        'sei':   [0.15,      0.20,      0.18,      0.25,      0.16,      0.22,      0.19],
        'year':  [2015,      2016,      2017,      2018,      2019,      2020,      2021],
    })
    toy.to_csv(csv_path, index=False)

csv_path

In [ ]:
df = pd.read_csv(csv_path)
required = {'study','yi','sei'}
missing = required - set(df.columns)
if missing:
    raise ValueError(f'Missing columns in CSV: {sorted(missing)}')

df = df.copy()
df['vi'] = df['sei'] ** 2
df['w_fe'] = 1.0 / df['vi']
df['ci_lo'] = df['yi'] - 1.96 * df['sei']
df['ci_hi'] = df['yi'] + 1.96 * df['sei']

def pooled_fixed(yi, vi):
    w = 1.0 / vi
    mu = np.sum(w * yi) / np.sum(w)
    se = np.sqrt(1.0 / np.sum(w))
    return mu, se

def heterogeneity_Q(yi, vi, mu_fe=None):
    if mu_fe is None:
        mu_fe, _ = pooled_fixed(yi, vi)
    w = 1.0 / vi
    Q = np.sum(w * (yi - mu_fe) ** 2)
    df_q = max(int(len(yi) - 1), 0)
    return Q, df_q

def tau2_dl(yi, vi):
    mu_fe, _ = pooled_fixed(yi, vi)
    Q, df_q = heterogeneity_Q(yi, vi, mu_fe)
    w = 1.0 / vi
    c = np.sum(w) - (np.sum(w**2) / np.sum(w))
    tau2 = max(0.0, (Q - df_q) / c) if c > 0 else 0.0
    return tau2, Q, df_q

def pooled_random(yi, vi):
    tau2, Q, df_q = tau2_dl(yi, vi)
    w = 1.0 / (vi + tau2)
    mu = np.sum(w * yi) / np.sum(w)
    se = np.sqrt(1.0 / np.sum(w))
    return mu, se, tau2, Q, df_q

yi = df['yi'].to_numpy(float)
vi = df['vi'].to_numpy(float)

mu_fe, se_fe = pooled_fixed(yi, vi)
mu_re, se_re, tau2, Q, df_q = pooled_random(yi, vi)
I2 = max(0.0, (Q - df_q) / Q) * 100.0 if Q > 0 else 0.0

summary = pd.DataFrame([
    {'model':'fixed',  'k': len(df), 'mu': mu_fe, 'se': se_fe, 'ci_lo': mu_fe-1.96*se_fe, 'ci_hi': mu_fe+1.96*se_fe, 'tau2': 0.0,  'Q': Q, 'df': df_q, 'I2_pct': I2},
    {'model':'random', 'k': len(df), 'mu': mu_re, 'se': se_re, 'ci_lo': mu_re-1.96*se_re, 'ci_hi': mu_re+1.96*se_re, 'tau2': tau2, 'Q': Q, 'df': df_q, 'I2_pct': I2},
])

df.head(), summary

In [ ]:
# Save summary table
summary_path = OUT_DIR / 'meta_summary.csv'
summary.to_csv(summary_path, index=False)

# Forest plot (log scale metric; also show exponentiated OR on right axis labels)
k = len(df)
ypos = np.arange(k, 0, -1)

fig_h = 0.55 * k + 2.8
fig, ax = plt.subplots(figsize=(8.8, fig_h))

# Study CIs
ax.hlines(ypos, df['ci_lo'], df['ci_hi'], color='black', lw=1)
sizes = 60 * (df['w_fe'] / df['w_fe'].max())
ax.scatter(df['yi'], ypos, s=sizes, color='black', zorder=3)

# Reference line at 0
ax.axvline(0.0, color='gray', lw=1, linestyle='--')

# Pooled effects (diamonds)
def diamond(ax, mu, lo, hi, y, height=0.25, **kwargs):
    xs = [lo, mu, hi, mu, lo]
    ys = [y,  y+height, y, y-height, y]
    ax.plot(xs, ys, **kwargs)

y_fe = 0.1
y_re = -0.6
diamond(ax, mu_fe, mu_fe-1.96*se_fe, mu_fe+1.96*se_fe, y_fe, color='tab:blue', lw=2)
diamond(ax, mu_re, mu_re-1.96*se_re, mu_re+1.96*se_re, y_re, color='tab:red', lw=2)

# Labels
ax.set_yticks(list(ypos) + [y_fe, y_re])
ax.set_yticklabels(list(df['study']) + ['Pooled (FE)', 'Pooled (RE)'])
ax.set_xlabel('Effect size (log OR)')

x_all = np.r_[df['ci_lo'].to_numpy(), df['ci_hi'].to_numpy(), [mu_fe-1.96*se_fe, mu_fe+1.96*se_fe, mu_re-1.96*se_re, mu_re+1.96*se_re]]
pad = 0.08 * (x_all.max() - x_all.min() + 1e-9)
ax.set_xlim(x_all.min() - pad, x_all.max() + pad)

title = f"Toy meta-analysis (k={k}) | tau²={tau2:.4f}, I²={I2:.1f}%"
ax.set_title(title)
ax.set_ylim(y_re - 1.0, k + 1.2)
ax.grid(axis='x', color='0.9', lw=0.8)

# Secondary axis for OR
sec = ax.secondary_xaxis('top', functions=(np.exp, np.log))
sec.set_xlabel('Odds Ratio (exp)')

forest_path = OUT_DIR / 'forest_plot.png'
fig.tight_layout()
fig.savefig(forest_path, dpi=200, bbox_inches='tight')
plt.close(fig)

forest_path, summary_path

In [ ]:
# Append run log
run_log_path = OUT_DIR / 'run_log.txt'
ts = datetime.now(timezone.utc).astimezone().isoformat(timespec='seconds')

log_lines = [
    f"[{ts}] meta_analysis_demo run",
    f"  csv: {csv_path}",
    f"  summary: {summary_path}",
    f"  forest_plot: {forest_path}",
    f"  fixed: mu={mu_fe:.4f}, se={se_fe:.4f}, ci=[{mu_fe-1.96*se_fe:.4f}, {mu_fe+1.96*se_fe:.4f}]",
    f"  random: mu={mu_re:.4f}, se={se_re:.4f}, ci=[{mu_re-1.96*se_re:.4f}, {mu_re+1.96*se_re:.4f}], tau2={tau2:.6f}",
    f"  heterogeneity: Q={Q:.4f}, df={df_q}, I2={I2:.1f}%",
    "",
]
with run_log_path.open('a', encoding='utf-8') as f:
    f.write("\n".join(log_lines))

run_log_path